In [1]:
import multiprocessing as mp
import netket as nk
import os
import json
from qutip import *
import numpy as np
import scipy
import cmath
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import itertools
from numpy.random import seed
from numpy.random import rand
from itertools import product
from scipy.misc import derivative
from scipy import optimize
from functools import wraps
from functools import reduce
from time import time
from scipy.optimize import minimize
from matplotlib import gridspec
import copy
import pickle
import optax
import excited 

In [5]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = excited.LocalOperator_e(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

#input_basis directly from output of exact_diagnolization, input_vec as np array object
def basis_trans(input_vec, input_basis, N):
    vec_new = np.linalg.inv(np.array(v).reshape(2**N, 2**N).T).dot(input_vec.reshape(2**N, 1))
    return vec_new


In [6]:
# functions for excited states

def CSHam_ex(N, B, Ak, state_list, Filling):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # now we add the extra gap filler
    fill_Op = scipy.sparse.csr_matrix((2**N, 2**N), dtype = 'complex128')
    for j in range(len(Filling)):
        fill_Op += Filling[j] * scipy.sparse.kron(state_list[j], scipy.sparse.spmatrix.getH(state_list[j])) 
        
    #operators.append(fill_Op)
    ex_site = np.arange(0,N).tolist()
    # Create hamiltonian
    hamiltonian = excited.LocalOperator_e(hilbertSpace, operators=operators, acting_on=sites, dtype='complex128')
    hamiltonian._add_operator(fill_Op, ex_site)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err_sta(state, edState,N):
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return waveFunctionErr

# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        #self.optimizer = nk.optimizer.Sgd(learning_rate=0.02,decay = 0.96)
        #self.optimizer = nk.optimizer.AdaGrad(learning_rate=0.3, epscut = 10**-2, initial_accumulator_value = 0.005)
        #self.optimizer = nk.optimizer.RmsProp(learning_rate=0.03, epscut = 10**-2, beta = 0.85)
        #self.optimizer = nk.optimizer.Momentum(learning_rate=0.1, nesterov = False, beta = 0.8)
        #self.optimizer = optax.adam(learning_rate=0.01, eps = 10**-2 ,b1=0.85,b2=0.99, eps_root=0.01)
        #self.optimizer = optax.fromage(0.005,0.0005)
        self.optimizer = optax.rmsprop(learning_rate = 0.01, decay = 0.95, eps = 10**-2, initial_scale = 0.01)
        
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.0001) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.variational.MCState(self.sampler, self.machine, n_samples=8000, n_discard=800) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output):
        self.vs.init_parameters(nk.nn.initializers.normal(stddev=0.25))
        gs = nk.VMC(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, precondioner=self.sr)
        # Set the output files as well as number of iterations in the descent
        gs.run(n_iter=10000, out=output)
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state


def runDescentCS_ex(N,B,Ak,alpha,state_list, Filling, i):
    np.random.seed()
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam_ex(N,B,Ak,state_list, Filling)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype='complex128',use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state = rbm("2021_summer_data/dynamics_ex_log/ex_con_Logs"+str(N)+str(i)) #where _call_ will be invoked
    return eng, state

def combined(N,B,Ak,alpha):
    eng = []
    state = []
    
    zeros = np.zeros(2**N)
    state_list_0 = [zeros, zeros, zeros, zeros]
    Filling_0 = [0, 0, 0, 0]
    
    e_0, v_0 = runDescentCS_ex(N, B, Ak, alpha, state_list_0, Filling_0, 0)
    fill = 2 * np.abs(e_0)
    eng.append(e_0)
    state.append(v_0)
    
    state_list_1 = [v_0, zeros, zeros, zeros]
    Filling_1 = [fill, 0, 0, 0]
    e_1, v_1 = runDescentCS_ex(N, B, Ak, alpha, state_list_1, Filling_1, 1)
    eng.append(e_1)
    state.append(v_1)
    
    state_list_2 = [v_0, v_1, zeros, zeros]
    Filling_2 = [fill, fill, 0, 0]
    e_2, v_2 = runDescentCS_ex(N, B, Ak, alpha, state_list_2, Filling_2, 2)
    eng.append(e_2)
    state.append(v_2)
    
    state_list_3 = [v_0, v_1, v_2, zeros]
    Filling_3 = [fill, fill, fill, 0]
    e_3, v_3 = runDescentCS_ex(N, B, Ak, alpha, state_list_3, Filling_3, 3)
    eng.append(e_3)
    state.append(v_3)
    
    state_list_4 = [v_0, v_1, v_2, v_3]
    Filling_4 = [fill, fill, fill, fill]
    e_4, v_4 = runDescentCS_ex(N, B, Ak, alpha, state_list_4, Filling_4, 4)
    eng.append(e_4)
    state.append(v_4)
    
    return eng, state

def state_error(state_list, ex_state_list, N):
    state_er = []
    for i in range(len(state_list)):
        state_er_temp = err_sta(state_list[i], ex_state_list[i], N)
        state_er.append(state_er_temp)
    return state_er

def eng_error(eng_list, ex_energy):
    eng_er = []
    for i in range(len(eng_list)):
        eng_er_temp = np.abs(eng_list[i] - ex_energy[i])
        eng_er.append(eng_er_temp)
    return eng_er

def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

In [7]:
N = 6
Ak = []

alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    #Ak_i = 1
    # Variable A
    Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)


#Exact Diagonalization
e, v = exactDiagonalization(ha)
print(e,v)
#eng_list, state_list = combined(N,B,Ak,alpha)

#print(state_error(state_list, v, N))
#print(eng_error(eng_list, e))

[-1.41524943] [array([-9.08381657e-17-1.97746318e-19j, -2.92931148e-02-4.14350458e-02j,
       -4.17620662e-02-5.90723499e-02j, -1.33471327e-16+3.56289706e-16j,
       -6.00892863e-02-8.49961621e-02j,  2.38046078e-17+5.21620995e-16j,
       -4.04269963e-16+3.75068592e-16j, -3.57762935e-17-1.70431231e-18j,
       -8.76510418e-02-1.23982204e-01j, -4.45100802e-17+6.69150133e-16j,
       -6.33749925e-16+4.62956402e-16j, -4.32502948e-17+3.53408570e-17j,
       -1.74387194e-16+5.91518277e-16j, -4.55422849e-17-1.81494566e-18j,
        7.53298862e-17-5.42992161e-17j, -1.45779110e-16+4.84885219e-17j,
       -1.30561056e-01-1.84678323e-01j,  1.90114982e-17+9.73913230e-16j,
       -8.65164430e-16+6.33028496e-16j, -1.11276276e-16+2.71544643e-17j,
       -1.58504036e-16+7.63880076e-16j, -8.03645405e-17-8.55949570e-17j,
        1.46870794e-17-1.26397843e-16j, -1.63360532e-16+4.81328257e-17j,
       -4.03905660e-16+7.04816591e-16j, -9.88590547e-17-4.86005770e-17j,
        8.62501610e-17-8.17671462e-1

In [8]:
import os
os.chdir('/home/victor/Dynamics')
os.getcwd()

'/home/victor/Dynamics'

In [20]:
eng_list = []
state_list = []

eng_list.append(np.loadtxt("2021_summer_data/0.01%_error_excited_eng.json"))
state_list.append(np.loadtxt("2021_summer_data/0.01%_error_excited_state.json", dtype = complex))
v_0 = state_list[0].reshape(2**N,1) 
e_0 = eng_list[0]


zeros = np.zeros(2**N)
state_list_0 = [zeros, zeros, zeros, zeros]
Filling_0 = [0, 0, 0, 0]
fill = 2 * np.abs(e_0)


print(err_sta(v_0,v[0],N))
print(np.abs(e_0-e[0]))



5.8602951919617396e-05
1.1820290633846042e-05


In [9]:
v_0 = np.array(v[0]).reshape(2**N,1)
v_0[np.abs(v_0) < 10**-10] = 0
v_0 = scipy.sparse.csr_matrix(v_0)

print(v_0)

  (1, 0)	(-0.02929311478093805-0.041435045846167425j)
  (2, 0)	(-0.04176206616367755-0.05907234990417227j)
  (4, 0)	(-0.06008928632290202-0.08499616214500733j)
  (8, 0)	(-0.08765104175369129-0.12398220403286941j)
  (16, 0)	(-0.13056105552690853-0.18467832328304584j)
  (32, 0)	(0.5498202012351714+0.7777194544075288j)


In [11]:
zeros = np.zeros(2**N)
fill = 2 * np.abs(e[0])

zeros = scipy.sparse.csr_matrix(zeros)
state_list_1 = [v_0, zeros, zeros, zeros]
Filling_1 = [fill, 0, 0, 0]
e_1, v_1 = runDescentCS_ex(N, B, Ak, alpha, state_list_1, Filling_1, 1)

  1%|          | 63/10000 [00:08<22:04,  7.50it/s, Energy=-1.061-0.003j ± 0.014 [σ²=1.532, R̂=1.0000]]    


KeyboardInterrupt: 

In [11]:
a = np.array([1+1j,2,3,4,10+2j]).reshape(5,1)
a[np.abs(a) < 4] = 0
print(a)

[[ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 4.+0.j]
 [10.+2.j]]


In [17]:
c = 2 * scipy.sparse.kron(v_0, scipy.sparse.spmatrix.getH(v_0))
print(c.getnnz())

169


In [ ]:

e, v = exactDiagonalization(ha)
